In [43]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

In [65]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [45]:
# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

In [73]:
from torch.utils import data
from PIL import Image
import pandas as pd
from torchvision import transforms

class CarDataset(data.Dataset):
    def __init__(self) -> None:
        super().__init__()
        self.data_dir = os.path.join(os.pardir, 'dataset', 'cars_images')
        self.filenames = os.listdir(self.data_dir)
        self.to_tensor = transforms.Compose([
            transforms.ToTensor()
        ]) 
    
    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, index):
        torch.zeros((360, 360))
        return self.to_tensor(Image.open(os.path.join(self.data_dir, self.filenames[index])).resize((256, 256), Image.BICUBIC))

car_dataset = CarDataset()
car_dataset

In [78]:
import matplotlib.pyplot as plt
car_data_loader = data.DataLoader(dataset=car_dataset, batch_size=32, shuffle=True)
for car in car_data_loader:
    print(car.shape)
    plt.imshow(car[0])
    break

ModuleNotFoundError: No module named 'matplotlib'

In [85]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=2, padding=1, *args, **kwargs) -> None:
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, **kwargs),
            nn.LeakyReLU(0.2)
        )
    
    def forward(self, x):
        x = self.layers(x)
        return x


In [97]:
class Discriminator(nn.Module):
    def __init__(self, in_channels, image_size, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        
        self.convs = nn.Sequential(
            ConvBlock(in_channels, 128),
            ConvBlock(128, 256),
            ConvBlock(256, 512),
            ConvBlock(512, 1024),
        )
        img_size = list(image_size)
        for _ in range(len(self.convs)):
            for idx in range(len(img_size)):
                img_size[idx] //= 2

        self.classify = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024 * img_size[0] * img_size[1], 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.convs(x)
        x = self.classify(x)
        return x

discriminator = Discriminator(3, (256, 256))

discriminator(torch.zeros((4, 3, 256, 256), dtype=torch.float)).shape

4
[16, 16]


torch.Size([4, 1])

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_size, image_size, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        
        self.convs = nn.Sequential(
            ConvBlock(latent_size, 1024),
            ConvBlock(1024, 512),
            ConvBlock(512, 256),
            ConvBlock(256, 3),
        )
        self.classify = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024 * image_size[0] * image_size[1], 1),
            nn.Sigmoid()
        )
        self.layers = nn.Sequential(
            nn.Linear(latent_size, )
        )
    
    def forward(self, x):
        x = self.convs(x)
        x = self.classify(x)
        return x